#### Solved: "fresh" learner vs "loaded" learner 

Differ in subsequent fitting operation due to random states of their DataLoaders performin `one_batch` on items. By setting `shuffle_train=False` when consturcting the dataloaders we match identical perf on second phase of fitting.

It seems difficult to re-initialize the random seed built into the DataLoader due to it's self-referential design. 

In [1]:
from fastai2.vision.all import *

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [2]:
path = untar_data(URLs.MNIST_TINY)

In [3]:
seed=42

In [4]:
set_seed(seed)
dl = ImageDataLoaders.from_folder(path,
                                  seed=seed,
                                  valid_pct=0.2,
                                  num_workers=0,
                                  shuffle_train=False
                                 )

In [5]:
set_seed(seed)
learn = cnn_learner(dl, resnet18)

In [6]:
set_seed(seed)
learn.fit_one_cycle(2)

epoch,train_loss,valid_loss,time
0,1.288761,0.394393,00:09
1,0.770088,0.442477,00:09


In [7]:
learn.save('saved-learner-4')

##### Load Model as `learn2`

In [8]:
set_seed(seed)
dl2 = ImageDataLoaders.from_folder(path,
                                  seed=seed,
                                  valid_pct=0.2,
                                  num_workers=0,
                                  shuffle_train=False
                                 )

In [9]:
set_seed(seed)
learn2 = cnn_learner(dl2, resnet18)

In [10]:
learn2.load('saved-learner-4')

##### Perform same training on "identical" model: `learn` and `learn2`

In [11]:
set_seed(seed)
learn.fine_tune(2)

epoch,train_loss,valid_loss,time
0,0.087027,0.422698,00:09


epoch,train_loss,valid_loss,time
0,0.141374,0.367039,00:18
1,0.107582,0.321651,00:19


In [12]:
set_seed(seed)
learn2.fine_tune(2)

epoch,train_loss,valid_loss,time
0,0.087027,0.422698,00:09


epoch,train_loss,valid_loss,time
0,0.141374,0.367039,00:17
1,0.107582,0.321651,00:16


##### Misc Debugging

In [14]:
learn.dls

In [15]:
dl

In [16]:
learn2.dls

In [17]:
dl2

In [19]:
dl2.train.items == dl.train.items

True

In [20]:
dl2.valid.items == dl.valid.items

True

In [58]:
x1, y1 = learn.dls[0].one_batch()
x2, y2 = learn2.dls[0].one_batch()

In [59]:
y2 == y1

TensorCategory([False, False,  True, False,  True, False, False, False, False, False,
        False, False, False,  True,  True,  True,  True,  True, False,  True,
        False,  True,  True,  True, False, False, False,  True, False, False,
         True, False,  True,  True,  True, False, False, False,  True, False,
         True, False, False,  True, False, False, False,  True, False, False,
        False, False, False, False, False, False, False, False, False,  True,
        False, False, False,  True])

In [53]:
x1, y1 = dl.one_batch()
x2, y2 = dl2.one_batch()

In [54]:
y2 == y1

TensorCategory([ True, False, False, False,  True, False, False, False,  True, False,
         True,  True,  True, False,  True,  True, False, False,  True, False,
         True, False, False, False, False,  True, False,  True, False,  True,
        False, False, False,  True, False, False,  True,  True,  True, False,
         True, False,  True,  True, False, False, False,  True, False,  True,
        False,  True,  True, False,  True,  True,  True,  True, False,  True,
         True, False,  True,  True])

In [21]:
sd1 = learn.model.state_dict()
sd2 = learn2.model.state_dict()

In [24]:
# sd1 == sd2
# cmp(s1,s2)

In [31]:
k = sd1.keys()

In [43]:
k

odict_keys(['0.0.weight', '0.1.weight', '0.1.bias', '0.1.running_mean', '0.1.running_var', '0.1.num_batches_tracked', '0.4.0.conv1.weight', '0.4.0.bn1.weight', '0.4.0.bn1.bias', '0.4.0.bn1.running_mean', '0.4.0.bn1.running_var', '0.4.0.bn1.num_batches_tracked', '0.4.0.conv2.weight', '0.4.0.bn2.weight', '0.4.0.bn2.bias', '0.4.0.bn2.running_mean', '0.4.0.bn2.running_var', '0.4.0.bn2.num_batches_tracked', '0.4.1.conv1.weight', '0.4.1.bn1.weight', '0.4.1.bn1.bias', '0.4.1.bn1.running_mean', '0.4.1.bn1.running_var', '0.4.1.bn1.num_batches_tracked', '0.4.1.conv2.weight', '0.4.1.bn2.weight', '0.4.1.bn2.bias', '0.4.1.bn2.running_mean', '0.4.1.bn2.running_var', '0.4.1.bn2.num_batches_tracked', '0.5.0.conv1.weight', '0.5.0.bn1.weight', '0.5.0.bn1.bias', '0.5.0.bn1.running_mean', '0.5.0.bn1.running_var', '0.5.0.bn1.num_batches_tracked', '0.5.0.conv2.weight', '0.5.0.bn2.weight', '0.5.0.bn2.bias', '0.5.0.bn2.running_mean', '0.5.0.bn2.running_var', '0.5.0.bn2.num_batches_tracked', '0.5.0.downsample.

In [44]:
elem = sd1.pop('0.0.weight')

In [48]:
elem.shape

torch.Size([64, 3, 7, 7])

In [47]:
# test_eq(learn.model.state_dict(), learn2.model.state_dict())

TypeError: 'odict_keys' object is not subscriptable

In [32]:
sd1[k[0]]

TypeError: 'odict_keys' object is not subscriptable